# Custom Aggregation

Ppndas groupby objects come with many built-in aggregate functions. These are all available as strings within the `agg` method. There are, of course, many other possible aggregations that are not directly available. It is possible to define your own customized aggregate function. These customized functions must return a single value. We begin by reading in the City of Houston dataset.

In [ ]:
import pandas as pd
import numpy as np
emp = pd.read_csv('../data/employee.csv')
emp.head(3)

### Writing your own custom aggregation function

Let's suppose you would like to know the difference between the max and min value of a column for each group. Pandas does not have an aggregate function built to do this. You will have to define this one yourself. 

Each customized aggregate function is defined just as a regular Python function with the `def` keyword. Each function is **implicitly** passed the aggregating column, of the group, as a **Series**. This means that all Series methods will work on the passed argument. The `min_max` function below takes one argument, `s`, which is a Series object. It returns the difference between the max and min values of that Series.

In [ ]:
def min_max(s):
    return s.max() - s.min()

## Using a customized aggregation function

Customized aggregation functions are used similarly to the built-in aggregation functions. When using them within the `agg` method, use the actual function object and not the string name. The following finds the difference between the maximum and minimum salaries for each department. 

In [ ]:
emp.groupby('dept').agg(min_max_salary_diff=('salary', min_max))

### Implicit passing of aggregation Series

The above `agg` method passed the `salary` column as a Series to our customized aggregation function, `min_max`, for each group. The parameter `s` takes on this Series. We say this is implicit, because we don't actually see the function executed. An **explicit** call to `min_max` would look like this:

In [ ]:
min_max(emp['salary'])

Our custom aggregation function is not passed the entire salary column as done above, but just the salary column for the current group. Let's manually recreate the grouping that pandas did by first getting the unique values of departments.

In [ ]:
unique_depts = emp['dept'].drop_duplicates()
unique_depts.values

We can loop through these departments and filter for just the salary of that department and pass that Series into the `min_max` function. The the results are then printed to the screen matching the above work done by pandas.

In [ ]:
for dept in unique_depts:
    salary_group = emp.query('dept == @dept')['salary']
    min_max_diff = min_max(salary_group)
    print(f'{dept:30} {min_max_diff}')

### Using an anonymous function

For custom aggregation functions that have just a single line of code with little logic, an anonymous function can be used. But, if there is complex logic and it is a function that you will reuse, then define a normal function. Here, we use an anonymous function to recreate the result of `min_max`. 

In [ ]:
emp.groupby('dept').agg(min_max_salary_diff=('salary', lambda s: s.max() - s.min()))

###  Custom aggregation function must return a single value

The custom aggregation function you define must return a single value, or else an exception will be raised. Let's create a custom aggregation that adds 5 to each value. This will return a Series the same size as the group and not a single value.

In [ ]:
def add5(s):
    return s + 5

Attempting to use this custom aggregation produces an error informing us that we did not produce an aggregated value.

In [ ]:
emp.groupby('dept').agg(salary_plus_5=('salary', add5))

### Combine custom aggregation function with built-ins
The custom aggregation function can be used in conjunction with any number of other built-in aggregation functions that we have previously seen. You will have to rename the columns to remove the MutliIndex as usual.

In [ ]:
emp.groupby(['dept', 'sex']) \
       .agg(min_salary=('salary', 'min'),
            max_salary=('salary', 'max'),
            min_max_salary_diff=('salary', min_max)) \
        .head(10).round(-3).style.format('{:,.0f}')

## Find the mean salary for the five highest paid employees per department

In this section, we will find the mean salary of the five highest paid employees per department. This again requires a custom aggregation function. The same Series of salaries will be implicitly passed to it. We use the `nlargest` method to select the top five salaries and then take the mean of them to return a single value.

In [ ]:
def top5_mean(s):
    return s.nlargest(5).mean()

Let's use this function to generate the desired result.

In [ ]:
emp.groupby('dept').agg(top5_average_salary=('salary', top5_mean)).round(-3)

### What percent of total salary do these five employees represent?

Let's take our question a step further and find out the percentage of the total salary from each department that these five employees represent. Instead of finding the mean, we take the sum of the top five salaries and divide it by the sum of all salaries for that department.

In [ ]:
def top5_percent(s):
    return s.nlargest(5).sum() / s.sum() * 100

Again, we pass our function to the `agg` method to get the result.

In [ ]:
emp.groupby('dept').agg(top5_average_salary=('salary', top5_percent)).round(2)

The above results tell us that the top five highest paid employees of the fire department represent 0.65% of the total combined fire department salary. Let's verify this by calculating it without using `groupby`.

In [ ]:
fire_sal_sorted = emp.query('dept == "Fire"')['salary'] \
                     .sort_values(ascending=False)
fire_sal_sorted.iloc[:5].sum() / fire_sal_sorted.sum() * 100

These numbers would be more meaningful if we compared them against the percentage of the number of employees that these five represent. For instance, if there are 200 fire department employees, then five would represent 2.5% (5 / 200). Another custom aggregation function is used below. Notice that the `count` method is used as several employees are missing salaries. If there are less than five employees in the group, 100% will be returned.

In [ ]:
def count5_percent(s):
    return min(5 / s.count(), 1) * 100

Let's use both custom aggregation functions.

In [ ]:
emp.groupby('dept').agg(top5_average_salary=('salary', top5_percent),
                        count5_percent=('salary', count5_percent)).round(2)

This gives us more meaning behind the results. The five fire department employees account for .11% of all employees, but .65% of the total salary. Since not all employees receive the same salary, we expect the top five to have a disproportional percentage of the total salary.

### Using a custom aggregation function in a pivot table

As we learned, pivot tables are nearly identical to groupby aggregations. pandas allows you to set the `aggfunc` of the `pivot_table` method to a custom aggregation function. It will implicitly pass the Series of the `values` column to the aggregation function. Here we call the `top5_percent` on the salary column for each unique combination of department and sex.

In [ ]:
emp.pivot_table(index='dept', columns='sex', 
                values='salary', aggfunc=top5_percent).round(2)

## Percentage of employees by department with salaries greater than 100,000

In this section, we'll find the percentage of employees by each department with salaries greater that 100,000 using a custom aggregation function. Let's define our function below.

In [ ]:
def perc_gt_100k(s):
    return (s > 100_000).mean() * 100

We use this custom function just like the others in this chapter to get our result.

In [ ]:
emp.groupby('dept').agg(percent_greater_100k=('salary', perc_gt_100k)).round(2)

### Not accounting for missing values

Some employees do not have salaries and when we make the comparison `s > 100_000`, those missing values will evaluate as `False`. Let's find the number of employees per department that have missing values with an anonymous custom function.

In [ ]:
emp.groupby('dept').agg(num_missing=('salary', lambda x: x.isna().sum()))

If we want to find the percentage of employees with known salary that have a salary greater than 100,000, then we'll have to either drop these missing values or use a denominator that is equivalent to the number of employees with known values. Below, we use the second option and create a new custom function that uses the `count` method to count the number of employees with known salary. Only the police department's returned statistic will change, as it was the only one that had missing values.

In [ ]:
def perc_gt_100k_real(s):
    return (s > 100_000).sum() / s.count() * 100

emp.groupby('dept').agg(percent_greater_100k=('salary', perc_gt_100k_real)).round(2)

## Optimizing a Custom Aggregation function

Custom aggregation functions have potential to be some of the slowest operations you'll perform. pandas optimizes its own built-in aggregations but cannot do the same for the ones you define. This section will show you how to optimize the performance of your custom aggregation function.

### Create a larger DataFrame to test performance

Our current DataFrame contains only 24,000 rows and performance metrics matter more with larger datasets. To simulate a larger dataset, we'll use the `sample` method to select 100,000 random rows with replacement.

In [ ]:
emp_large = emp.sample(100_000, replace=True).reset_index(drop=True)
emp_large.head(3)

Let's verify that we have a DataFrame with 100,000 rows.

In [ ]:
emp_large.shape

Let's measure the performance of one of custom aggregation functions, `perc_gt_100k`, with this larger dataset.

In [ ]:
%timeit -r 1 -n 5 emp_large.groupby('dept').agg({'salary': perc_gt_100k})

The number of groups can greatly impact the performance of an operation. Let's take a look at the number of unique values in each column.

In [ ]:
emp_large.nunique()

There are 690 unique titles in our dataset. Let's use this column as the grouping column instead of department to show the difference in performance.

In [ ]:
%timeit -r 1 -n 5 emp_large.groupby('title').agg({'salary': perc_gt_100k})

On my machine, grouping by title was about 15 times slower, showcasing the vast differences in performance when grouping by columns with different numbers of groups.

### Convert grouping columns to categorical

The first step to help making your custom aggregation (and any aggregation) faster is to convert the grouping column to categorical if it is an object column containing strings. Grouping columns tend to be strings so it is a change that you'll do often. We create two new category columns for department and title and append them to the end of our DataFrame. When performing this on your own data, overwrite the original columns. We keep the original columns in this instance to show the difference in performance.

In [ ]:
emp_large[['dept_cat', 'title_cat']] = emp_large[['dept', 'title']].astype('category')

Before testing our custom function, let's compare the performance difference of built-in `mean` function between the two versions of the department column. Grouping with the categorical column reduced the run time by two-thirds.

In [ ]:
%timeit -r 1 -n 5 emp_large.groupby('dept').agg({'salary': 'mean'})

In [ ]:
%timeit -r 1 -n 5 emp_large.groupby('dept_cat').agg({'salary': 'mean'})

Let's run this comparison using title as the grouping column. A similar performance improvement is seen.

In [ ]:
%timeit -r 1 -n 5 emp_large.groupby('title').agg({'salary': 'mean'})

In [ ]:
%timeit -r 1 -n 5 emp_large.groupby('title_cat').agg({'salary': 'mean'})

Above, we timed our performance with the `perc_gt_100k` custom aggregation function. Let's redo this operation using the new categorical columns. Run time decreased around 50% with department, but stayed the same for title.

In [ ]:
%timeit -r 1 -n 5 emp_large.groupby('dept_cat').agg({'salary': perc_gt_100k})

In [ ]:
%timeit -r 1 -n 5 emp_large.groupby('title_cat').agg({'salary': perc_gt_100k})

### Complete operations that are independent of the group outside of the custom function

The next step to optimizing your custom function is to look for operations that are independent of the group. The `perc_gt_100k` function is simple and has only one line of code in its body. Let's examine the part of this line that tests whether each value is greater than 100,000.

```python
s > 100_000
```

This operation is **independent** of the group. Specifically, this operation does not change whether the group is 'Fire', 'Police', 'Library', or any of the other groups. Since there is no dependency on the group, we can compute it on the entire DataFrame first, outside of the custom aggregation function. We store the results as a new column in our DataFrame.

In [ ]:
emp_large['gt_100k'] = emp_large['salary'] > 100_000
emp_large.head(3)

### Use built-in aggregations if possible

We can now use the built-int `mean` aggregation function on the 'gt_100k' column to get our desired result. There is no need for the custom function anymore. Note that the results are likely to be different compared to above because `emp_large` was built from random rows.

In [ ]:
emp_large.groupby('dept_cat').agg({'gt_100k': 'mean'}).round(3) * 100

Built-in aggregation functions are optimized and should perform faster than custom aggregation functions doing similar tasks. Let's calculate the run time for this updated procedure taking into account the addition of the new column.

In [ ]:
%%timeit -r 1 -n 5
emp_large['gt_100k'] = emp_large['salary'] > 100_000
emp_large.groupby('dept_cat').agg({'gt_100k': 'mean'}).round(3) * 100

A modest (approximately 20%) performance gain is seen when grouping by department. Let's measure performance when grouping by title.

In [ ]:
%%timeit -r 1 -n 5
emp_large['gt_100k'] = emp_large['salary'] > 100_000
emp_large.groupby('title_cat').agg({'gt_100k': 'mean'}).round(3) * 100

Remarkably, the run time is nearly the same when grouping by title as it is when grouping by department and an astounding 20-30 times faster than the custom aggregation function.

### Use alternative syntax

As we've learned, there are a variety of groupby aggregation syntaxes and for whatever reason pandas has implemented them differently. The following syntax where the aggregating column appears to be selected as a Series after the call to `groupby` has the best performance. Using this alternative syntax improves our performance 60x over the original.

```python
df.groupby('grouping column')['aggregating column'].agg_func()
```

In [ ]:
%%timeit -r 1 -n 5 
emp_large['gt_100k'] = emp_large['salary'] > 100_000
emp_large.groupby('title_cat')['gt_100k'].mean()

### Using multiple built-in groupby aggregations

To optimize performance, you should avoid custom aggregation functions altogether and opt for those built-in to pandas. In some cases, it may be necessary to use multiple groupby aggregations that use built-in pandas functions to avoid a single custom one. Take a look at the main portion of the `perc_gt_100k_real` function, which takes into account missing values.

```python
(s > 100_000).sum() / s.count()
```

There are two aggregations, `sum` and `count`, which makes it appear that a custom aggregation will be necessary. Instead, we can use two groupby operations with built-in aggregation functions and then divide the results. Let's first calculate the total number of employees for each department with salaries greater than 100,000. You'll need to use the syntax that returns the result as a Series.

In [ ]:
num_gt_100k = emp_large.groupby('dept_cat')['gt_100k'].sum()
num_gt_100k.head(3)

Similarly, we find the number of employees that have non-missing values for salary.

In [ ]:
num_has_salary = emp_large.groupby('dept_cat')['salary'].count()
num_has_salary.head(3)

We can then divide these two Series together to get the desired result.

In [ ]:
(num_gt_100k / num_has_salary).round(3) * 100

Let's measure the performance of this strategy using title as the grouping column and include the time it takes to create the boolean 'gt_100k' column. An extra optimization is done, by assigning the initial grouping to variable `g`, as it is redundant to calculate it twice.

In [ ]:
%%timeit -r 1 -n 5
emp_large['gt_100k'] = emp_large['salary'] > 100_000
g = emp_large.groupby('title_cat')
num_gt_100k = g['gt_100k'].sum()
num_has_salary = g['salary'].count()
(num_gt_100k / num_has_salary).round(3) * 100

We can compare that result to the performance of our custom aggregation function. The custom aggregation function performs a massive 50x worse.

In [ ]:
%timeit -r 1 -n 5 emp_large.groupby('title_cat')['salary'].agg(perc_gt_100k_real)

Instead of using multiple calls to groupby, we can instead fill in missing values in 'gt_100k' where there are missing values for salary. This allows us to use just the `mean` aggregation. The performance is slightly worse than with the multiple groupby calls.

In [ ]:
%%timeit -n 5
emp_large['gt_100k'] = emp_large['salary'] > 100_000
filt = emp_large['salary'].isna()
emp_large.loc[filt, 'gt_100k'] = np.nan
emp_large.groupby('title_cat')['gt_100k'].mean()

## Summary of Custom Aggregation Functions

* Convert grouping columns to categorical
* Complete operations that are independent of the group outside of the custom function 
* Use built-in aggregations if possible
* Use alternative groupby syntax
* Use multiple built-in groupby aggregations if necessary

### Complexity vs Performance

This is usually a topic of debate when deciding on which Pandas methods to use. I typically like to avoid custom aggregation functions at all cost as they can drastically reduce performance for larger datasets.

Readability (low complexity) is very valuable when sharing your code or looking back at it at a later date. Custom aggregation function might provide more readability, but usually it isn't that much, so I tend to rarely use them as the performance difference is much more significant.

## Exercises

Execute the cell below to read in the flights dataset and then use it for the following exercises.

In [ ]:
import pandas as pd
flights = pd.read_csv('../data/flights.csv', parse_dates=['date'])
flights.head(3)

### Exercise 1

<span style="color:green; font-size:16px">What are the three airlines with the least number of flights?</span>

### Exercise 2

<span style="color:green; font-size:16px">For each airline, find the 75th percentile of flight distance. Use a custom aggregation function.</span>

### Exercise 3

<span style="color:green; font-size:16px">For each airline, find out what percentage of its flights leave on a Tuesday. Use a custom aggregation function.</span>

### Exercise 4

<span style="color:green; font-size:16px">Optimize exercise 2 without using a custom aggregation. What is the performance difference?</span>

### Exercise 5

<span style="color:green; font-size:16px">The range of salaries per department was calculated using the `min_max` custom function from the beginning of this chapter. Use this same function to calculate the range of distance for each airline. Then calculate this range again without a custom function.</span>

### Exercise 6

<span style="color:green; font-size:16px">Which origin airport has the highest percentage of its flights cancelled?</span>

### Use the college dataset

Execute the following cell which reads in a few columns from the college dataset, sets the institution name as the index and converts 'stabbr' and 'relaffil' to categorical.

In [ ]:
cols = ['instnm', 'stabbr', 'relaffil', 'satvrmid', 'satmtmid', 'ugds']
college = pd.read_csv('../data/college.csv', usecols=cols, 
                      index_col='instnm', dtype={'stabbr': 'category', 
                                                 'relaffil': 'category'})
college.head(3)

### Exercise 7

<span style="color:green; font-size:16px">How many states have more schools with a higher 'satvrmid' than 'satmtmid'? Make sure to not count schools that have missing values for either one.</span>

### Exercise 8

<span style="color:green; font-size:16px">Create a pivot table that shows the percentage of schools with less than 1,000 students in each state by religious affiliation. Also return the count of schools.</span>